In [17]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm, colors

import glob
import gc
import warnings
import copy
import pathlib
import pickle

import os
import shutil

from scipy import stats
import signac
import freud
import gsd.hoomd
import hoomd
import schmeud
import schmeud._schmeud as schmeud_rs
from schmeud._schmeud import statics
from schmeud import ml
from tqdm import tqdm

from dataclasses import dataclass
from collections import defaultdict

from numba import njit

In [18]:
from monk import nb, prep, pair, render, utils, workflow, grid

parent = pathlib.Path(os.getcwd()).parent.parent / "config.yaml"
config = workflow.get_config(parent.as_posix())
parent, config

(PosixPath('/home/ian/Projects/work/monk/workflows/2d-osc-shear/config.yaml'),
 {'root': '/media/ian/Data2/monk/2d-osc-shear',
  'origin': '/media/ian/Data2/monk/2d-esl'})

In [19]:
project: signac.Project = signac.get_project(root=config['root'])
project.doc

{'avail_seed': 18, 'dt': 0.005, 'step_unit': 200, 'equil_time': 100, 'min_periods': 20, 'dumps': 40, 'period_times': [30.0, 100.0, 300.0, 1000.0], 'max_shears': [0.01, 0.02, 0.03, 0.05, 0.08, 0.12, 0.04, 0.06, 0.07], '_status': {}}

In [20]:
@dataclass(frozen=True, eq=True)
class Statepoint:
    max_shear: float
    period: float
    temp: float
    prep: str
    iter: int

In [93]:
output = defaultdict(list)


for job in project:
    print(job)
    prep = job.sp["prep"]
    print(prep)
    
    experiments = sorted(glob.glob(job.fn("no_shear_short_experiments/*/*/traj-fire_period-*.gsd")))
    if len(experiments) == 0:
        continue
    for exper in experiments:
        max_shear = utils.extract_between(exper, "max-shear-", "/")
        period = utils.extract_between(exper, "period-", "_")
        temp = utils.extract_between(exper, "temp-", "/")
        iter = utils.extract_between(exper, "iter-", ".gsd")
        df_path = exper.replace("traj-fire", "softness").replace(".gsd", ".parquet")
        sp = Statepoint(max_shear=float(max_shear), period=float(period), temp=float(temp), prep=prep, iter=int(iter))
        

        if job.isfile(df_path):
            dataset = pl.read_parquet(job.fn(df_path), use_pyarrow=True)[::10].clone()
            if "phop" in dataset.columns:
                output[sp].append(dataset)
            else:
                print("No phop in", df_path)
            continue
    # break


634e14bc75836df75158fb15f379d8bb
HTL
7ec2f6aca035e69077eab24383a21cb6
ESL
cfa1e4c0007fd65d020f809715a6f0b4
HTL
ecce68c50e28a33684826f28780bf6e9
HTL
1d0fe2ba44a6f635f5504dcbcee0ebc9
ESL
1083a2a09eec2c6cf215ebc7a9706cac
HTL
09d3dccc56f1fa9330b57901cd232ab7
ESL
6976a7c10ada4159010403f8027f3ea5
HTL
9158e73b2e4c9fd1a565e1e5d1bc3af5
HTL
c6d7afd0de62bfa16d05da47815227eb
ESL
bf92a185d043ce3ef666e05e427c7e76
ESL
a8f2c249ed85533b56b0729b7ab96d18
ESL


In [80]:
temps = []

for sp, datasets in output.items():
    # print(sp.prep)
    temps.append(sp.temp)

temps = sorted(np.unique(temps))

HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
HTL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL
ESL


In [81]:
temps

[0.049591, 0.099182, 0.14877, 0.19836, 0.3, 0.4]

In [92]:
sbins = np.linspace(-1, 1, 11)
labels = ["-inf"]
mids = [f"{(sbins[i] + sbins[i+1])/2:.2}" for i in range(len(sbins)-1)]
labels.extend(mids)
labels.append("inf")

fig, axs = plt.subplots(2, 3, figsize=(12, 8), sharex=True, sharey=True)
cmap = plt.get_cmap("viridis")
norm = colors.Normalize(vmin=0, vmax=5)

axs = axs.ravel()

for key, val in output.items():
    # print(key, len(val))
    # print(key)
    # print(key)

    idx = temps.index(key.temp)

    plt.sca(axs[idx])
    c = cmap(norm(key.iter))

    if key.prep == "ESL":
        ls = "-"
    else:
        ls = ":"
    # print(sp.prep)

    

    df = pl.concat(val).explode(["soft", "phop", "id"])

    cuts = df["soft"].cut(sbins, labels).alias("cut")
    rearrange = (df["phop"] > 0.2).alias("rearrange")
    
    df = df.hstack([cuts, rearrange])
    out = df.group_by("cut", maintain_order=True).agg(pl.mean("rearrange").alias("mean_rearrange"))
    cut_float = out["cut"].apply(lambda x: float(x)).alias("cut_float")

    out = out.hstack([cut_float])
    out = out.sort(pl.col("cut_float"))

    x = out["cut_float"].to_numpy()[1:-1]
    y = out["mean_rearrange"].to_numpy()[1:-1]

    plt.plot(x, y, color=c, linestyle=ls)
    
    # break
    plt.yscale('log')
    plt.ylabel(r"$P_R$", size="xx-large")
    plt.xlabel(r"S", size="xx-large")

/tmp/ipykernel_871695/2012306676.py:33: DeprecationWarning: All arguments of Series.cut except for 'breaks' will be keyword-only in the next breaking release. Use keyword arguments to silence this warning.
  cuts = df["soft"].cut(sbins, labels).alias("cut")
/tmp/ipykernel_871695/2012306676.py:38: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  cut_float = out["cut"].apply(lambda x: float(x)).alias("cut_float")
/tmp/ipykernel_871695/2012306676.py:38: PolarsInefficientMapWarning: 
Series.map_elements is significantly slower than the native series API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - s.map_elements(lambda x: ...)
  + s.cast(pl.Float64)

  cut_float = out["cut"].apply(lambda x: float(x)).alias("cut_float")


In [88]:
cut_float

shape: (12,)
Series: 'cut_float' [f32]
[
	0.0
	1.0
	2.0
	3.0
	4.0
	5.0
	6.0
	7.0
	8.0
	9.0
	10.0
	11.0
]

/tmp/ipykernel_871695/2827598458.py:1: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  cut_float = out["cut"].apply(lambda x: float(x)).alias("cut_float")
/tmp/ipykernel_871695/2827598458.py:1: PolarsInefficientMapWarning: 
Series.map_elements is significantly slower than the native series API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - s.map_elements(lambda x: ...)
  + s.cast(pl.Float64)

  cut_float = out["cut"].apply(lambda x: float(x)).alias("cut_float")


In [63]:
out.sort(pl.col("cut_float"))

shape: (12, 3)
┌──────┬────────────────┬───────────┐
│ cut  ┆ mean_rearrange ┆ cut_float │
│ ---  ┆ ---            ┆ ---       │
│ cat  ┆ f64            ┆ f64       │
╞══════╪════════════════╪═══════════╡
│ -inf ┆ 0.000921       ┆ -inf      │
│ -0.9 ┆ 0.001044       ┆ -0.9      │
│ -0.7 ┆ 0.000965       ┆ -0.7      │
│ -0.5 ┆ 0.001095       ┆ -0.5      │
│ …    ┆ …              ┆ …         │
│ 0.5  ┆ 0.00149        ┆ 0.5       │
│ 0.7  ┆ 0.001627       ┆ 0.7       │
│ 0.9  ┆ 0.002021       ┆ 0.9       │
│ inf  ┆ 0.003202       ┆ inf       │
└──────┴────────────────┴───────────┘

In [39]:
df["rearrange"].mean()

0.001337432861328125

In [7]:
cmap = cm.magma
norm = colors.Normalize(vmin=0, vmax=0.15)

fig, axs = plt.subplots(1, 5, figsize=(20, 4), sharex=True, sharey=True)

for key, value in output.items():
    if key.prep == "HTL":
        ln = ":"
    else:
        ln = "-"
    
    dataset = pl.concat(value)
    xs = []
    ys = []df.head()

    temps = [0.019836, 0.049591, 0.099182, 0.14877, 0.19836]
    i = temps.index(key.temp)
    ax = axs[i]
    
    for frame, df in dataset.group_by("frame", maintain_order=True):
        # print(frame)
        df = df.explode(["id", "soft"])
        mean_soft = df["soft"].mean()

        xs.append(frame/40)
        ys.append(mean_soft)
    ax.plot(xs, ys, linestyle=ln, color=cmap(norm(key.max_shear)), label=f"{key.prep} {key.max_shear:.2f}")

for ax in axs:
    ax.set_xlabel("Cycle")
    ax.set_ylabel("Softness")

KeyboardInterrupt: 